## 各种文件格式转换

#### KML转geojson/shapefile

In [1]:
from osgeo import ogr, gdal
import os

# 定义函数

def convert_kml_to_format(input_file, output_file, output_format="GeoJSON"):
    """
    Converts a KML file to another geospatial data format.

    Args:
    input_file (str): Path to the input KML file.
    output_file (str): Path where the output file will be saved.
    output_format (str): Format of the output file. Default is "GeoJSON".
                         Other formats could be "ESRI Shapefile", etc.
    """
    # 获取输入文件的数据源
    input_ds = ogr.Open(input_file)
    if input_ds is None:
        raise RuntimeError(f"Failed to open input file: {input_file}")

    # 获取驱动程序，用于输出文件的格式
    driver = ogr.GetDriverByName(output_format)
    if driver is None:
        raise RuntimeError(f"Driver not found for format: {output_format}")

    # 如果输出文件已存在，先删除它
    if ogr.Open(output_file) is not None:
        driver.DeleteDataSource(output_file)
    
    # 创建输出文件的数据源
    output_ds = driver.CreateDataSource(output_file)
    if output_ds is None:
        raise RuntimeError(f"Failed to create output file: {output_file}")

    # 遍历输入数据源中的每个图层并复制到输出数据源
    for i in range(input_ds.GetLayerCount()):
        input_layer = input_ds.GetLayerByIndex(i)
        output_layer = output_ds.CreateLayer(input_layer.GetName(),
                                             geom_type=input_layer.GetGeomType())
        # 复制图层定义
        output_layer_defn = output_layer.GetLayerDefn()

        # 从输入图层复制要素到输出图层
        for feature in input_layer:
            output_feature = ogr.Feature(output_layer_defn)
            output_feature.SetGeometry(feature.GetGeometryRef().Clone())
            output_layer.CreateFeature(output_feature)

    # 清理资源
    del output_ds, input_ds


In [6]:

# 调用函数进行转换
input_file = '../Data/Transport/Cycling/CycleRoutes.kml'
output_file = '../Data/Transport/Cycling/CycleRoutes.geojson'
convert_kml_to_format(input_file, output_file, "GeoJSON")  # 或者 "ESRI Shapefile" 为Shapefile格式




#### JSON转geojson/shapefile

In [4]:
import ijson

In [5]:
# 分析json文件的结构

def print_json_structure(filename):
    """
    使用ijson打印JSON文件的结构。
    """
    with open(filename, 'rb') as file:
        # 使用ijson解析文件结构
        parser = ijson.parse(file)
        path = []
        for prefix, event, value in parser:
            if event == 'start_map':
                path.append(prefix.split('.')[-1] if prefix else 'root')
                print(' -> '.join(path))
            elif event == 'end_map':
                path.pop()


In [16]:
# 读取大型JSON文件并另存为geojson格式
import json
from geojson import Feature, FeatureCollection, LineString, Point

def convert_json_to_geojson(input_file, output_file):
    # 读取JSON文件
    with open(input_file, 'r') as file:
        data = json.load(file)
    
    
    # 解析features数组，构建GeoJSON特征列表
    features = []
    for feature in data['features']:
        geometry = feature['geometry']
        properties = feature['properties']
        
        # 根据geometry类型创建相应的GeoJSON geometry对象
        if geometry['type'] == 'LineString':
            geojson_geom = LineString(geometry['coordinates'])
        elif geometry['type'] == 'Point':
            geojson_geom = Point(geometry['coordinates'])
        else:
            # 如果有其他未支持的几何类型，可以在这里扩展
            continue  # 跳过当前循环，处理下一个feature
        
        # 创建并添加GeoJSON Feature
        geojson_feature = Feature(geometry=geojson_geom, properties=properties)
        features.append(geojson_feature)
    
    # 创建GeoJSON FeatureCollection
    feature_collection = FeatureCollection(features)
    
    # 保存GeoJSON文件
    with open(output_file, 'w') as file:
        json.dump(feature_collection, file, indent=2)
    
    print(f"GeoJSON file has been saved as: {output_file}")


In [25]:
# 示例用法
input_file = '../Data/Transport/Cycling/JSONorKML/traffic_calming.json'  # 更换为你的JSON文件路径
output_file = '../Data/Transport/Cycling/GEOJSON/traffic_calming.geojson'  # 更换为输出文件路径
convert_json_to_geojson(input_file,output_file)

GeoJSON file has been saved as: ../Data/Transport/Cycling/GEOJSON/traffic_calming.geojson


## XML转geojson

In [ ]:
import xml.etree.ElementTree as ET
import json

In [ ]:


# 解析XML
root = ET.fromstring("../Data/AirQuality/MonitoringSites.xml")

# 创建GeoJSON结构
geojson = {
    "type": "FeatureCollection",
    "features": []
}

for site in root.findall('Site'):
    feature = {
        "type": "Feature",
        "properties": {
            "LocalAuthorityCode": site.attrib['LocalAuthorityCode'],
            "LocalAuthorityName": site.attrib['LocalAuthorityName'],
            "SiteCode": site.attrib['SiteCode'],
            "SiteName": site.attrib['SiteName'],
            "SiteType": site.attrib['SiteType'],
            "DateClosed": site.attrib['DateClosed'],
            "DateOpened": site.attrib['DateOpened'],
            "DataOwner": site.attrib['DataOwner'],
            "DataManager": site.attrib['DataManager'],
            "SiteLink": site.attrib['SiteLink'],
            "Species": []
        },
        "geometry": {
            "type": "Point",
            "coordinates": [float(site.attrib['Longitude']), float(site.attrib['Latitude'])]
        }
    }
    
    # 添加物种信息到properties
    for species in site.findall('Species'):
        feature["properties"]["Species"].append({
            "SpeciesCode": species.attrib['SpeciesCode'],
            "SpeciesDescription": species.attrib['SpeciesDescription'],
            "DateMeasurementStarted": species.attrib['DateMeasurementStarted'],
            "DateMeasurementFinished": species.attrib['DateMeasurementFinished'],
        })
    
    # 将feature添加到features列表中
    geojson["features"].append(feature)

# 将GeoJSON转换为字符串
geojson_str = json.dumps(geojson, indent=4)

# 打印GeoJSON字符串
print(geojson_str)